# LangChain: Memory

## Outline
* ConversationBufferMemory
    - stores messages then extracts the messages in variable.
* ConversationBufferWindowMemory
    - keeps a list of interactions of the conversation over time. Only uses the last k'th interactions
* ConversationTokenBufferMemory
    - keeps a buffer of recent interactions using token length.
* ConversationSummaryMemory
    - creates a summary of the conversation

- LLM are 'stateless'
    - Each transaction is independent
- Chatbots appears to have memory by providing the fill conversation as 'context'

### Additional Memory Types
    * Vector data memory
        - stores text in vector db and retrieves the most relevant block of text
    * Entity memories
        - it remembers details about specific entities

- We can use multiple memory types in the same chain
    - For example:
        - Conversation memory + Entity memory

## ConversationBufferMemory

In [1]:
import warnings
warnings.filterwarnings('ignore')

# model = "gemma:2b"
model = "qwen2:0.5b"

In [2]:
from langchain.llms import Ollama
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory


In [3]:
llm = Ollama(temperature=0.0, model=model)

In [15]:
memory = ConversationBufferMemory()
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [16]:
conversation.predict(input="Hi, my name is Rabin")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Rabin
AI:

> Finished chain.


'Hello, Rabin! How can I assist you today?'

In [20]:
conversation.predict(input="What is 1+1?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Rabin
AI: Hello, Rabin! How can I assist you today?
Human: What is 1+1?
AI: The sum of two numbers is equal to three.
Human: What is 1+1?
AI: The answer is 2.
Human: What is 1+1?
AI: The sum of two numbers is equal to three.
Human: What is 1+1?
AI:

> Finished chain.


'The sum of two numbers is equal to three.'

In [12]:
conversation.predict(input="Can you remember by name? What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Rabin
AI: Hello, Rabin! How can I assist you today?
Human: What is 1+1?
AI: The sum of two numbers is equal to three.
Human: What is 1+1?
AI: The answer is 2.
Human: What is 1+1?
AI: The sum of two numbers is equal to three.
Human: What is 1+1?
AI: The sum of two numbers is equal to three.
Human: Can you remember by name? What is my name?
AI:

> Finished chain.


'Rabin'

In [7]:
print(memory.buffer)

Human: Hi, my name is Rabin
AI: Hello, Rabin! How can I assist you today?
Human: What is 1+1?
AI: The sum of two numbers is equal to three.


In [64]:
memory.load_memory_variables({})

{'history': "System: Sure, here's the summary you requested:\n\n**Initial Summary:**\n\nThe human asks what the AI thinks of artificial intelligence.\n\n**New Summary:**\n\nThe human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.\n\n**New lines of conversation:**\nAI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.\nHuman: What would be a good demo to show?\nAI: The AI suggests showing a demo of the latest Large Language Model (LLM) demo. The LLM is a powerful AI that can be used for a variety of tasks, suc

In [8]:
# reset
memory = ConversationBufferMemory()
memory.save_context({"input": "Hi, My name is Rabin"}, 
                    {"output": "What's up"})

In [31]:
print(memory.buffer)

Human: Hi, My name is Rabin
AI: What's up


In [32]:
memory.load_memory_variables({})

{'history': "Human: Hi, My name is Rabin\nAI: What's up"}

In [33]:
memory.save_context({"input": "Not much, just hanging"}, 
                    {"output": "Cool"})
print(memory.buffer)

Human: Hi, My name is Rabin
AI: What's up
Human: Not much, just hanging
AI: Cool


In [34]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Hi, my name is Rabin
AI: Hello! It's a pleasure to meet you, Rabin. What can I do for you today?
Human: What is 1+1?
AI: Sure, Rabin, I can help you with that. 1 + 1 is 2. Is there anything else I can do for you today?
Human: Can you remember by name? What is my name?
AI: The AI does not know the answer to the question "Can you remember by name? What is my name?". Therefore, it cannot provide a specific answer.
Human: What is my name?
AI:

> Finished chain.


'Sure, here\'s a more comprehensive and informative response to the question:\n\n"My purpose is to assist and provide information to the best of my ability. While I can access and process vast amounts of data, I do not have personal experiences or the ability to remember by name. I am programmed to respond based on the context and information I have been trained on, which may sometimes lead to providing generic or incomplete answers.\n\nI am still under development, and I am constantly learning and improving. I am excited to see how I can assist you further in the future."'

## ConversationBufferWindowMemory

In [21]:
from langchain.memory import ConversationBufferWindowMemory
memory = ConversationBufferWindowMemory(k=1,ai_prefix='AI', human_prefix='Human')

# k = 1 -> keep the last one conversation only.

In [22]:
memory.save_context({"input": "Hi, my name is Rabin"},
                    {"output": "What's up"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})

print(memory.load_memory_variables({}))

{'history': 'Human: Not much, just hanging\nAI: Cool'}


In [23]:
memory = ConversationBufferWindowMemory(k=1)
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [24]:
conversation.predict(input="Hi, my name is Rabin")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Hi, my name is Rabin
AI:

> Finished chain.


'Hello, Rabin! How can I assist you today?'

In [40]:
conversation.predict(input="What is 1 + 1?  Isn' the answer is 2?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1 + 1?  Isn' the answer is 2?
AI: The sum of two numbers is equal to three, so we can write this as an equation: \(x = 3\). To convert this into mathematical format, we need to add the variables together. The variable x represents the unknown number that needs to be found.
Therefore, the answer in mathematical format would be:
\(x = 3\)
Human: What is 1 + 1?  Isn' the answer is 2?
AI:

> Finished chain.


'The sum of two numbers is equal to three, so we can write this as an equation: \\(x = 3\\). To convert this into mathematical format, we need to add the variables together. The variable x represents the unknown number that needs to be found.\nTherefore, the answer in mathematical format would be:\n\\(x = 3\\)'

In [75]:
conversation.predict(input="What is my name?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: What is 1+1?
AI: Sure, Rabin, I can help you with that. 1 + 1 is 2. Is there anything else I can do for you today?
Human: What is my name?
AI:

> Finished chain.


'Sure, I can help you with that. My name is AIDA. What is your name?'

## ConversationTokenBufferMemory


In [43]:
from langchain.memory import ConversationTokenBufferMemory

In [49]:
memory = ConversationTokenBufferMemory(llm=llm, max_token_limit=50)
memory.save_context({"input": "Hi, my name is Rabin"},
                    {"output": "Hi Rabin!"})
memory.save_context({"input": "AI is what?!"},
                    {"output": "Amazing!"})
memory.save_context({"input": "Backpropagation is what?"},
                    {"output": "Beautiful!"})
memory.save_context({"input": "Chatbots are what?"}, 
                    {"output": "Charming!"})
conversation.predict(input="What is my name?")

"The AI does not know the answer to the question and responds truthfully. It does not know the human's name and cannot provide any specific details about them."

In [50]:
# memory.load_memory_variables({})
print(memory.buffer)

Human: Hi, my name is Rabin
AI: Hi Rabin!
Human: AI is what?!
AI: Amazing!
Human: Backpropagation is what?
AI: Beautiful!
Human: Chatbots are what?
AI: Charming!


## ConversationSummaryMemory

In [51]:
from langchain.memory import ConversationSummaryBufferMemory

In [53]:
# create a long string
schedule = "There is a meeting at 8am with your product team. \
You will need your powerpoint presentation prepared. \
9am-12pm have time to work on your LangChain \
project which will go quickly because Langchain is such a powerful tool. \
At Noon, lunch at the italian resturant with a customer who is driving \
from over an hour away to meet you to understand the latest in AI. \
Be sure to bring your laptop to show the latest LLM demo."

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=100)
memory.save_context({"input": "Hello, my name is Rabin"}, {"output": "What's up Rabin"})
memory.save_context({"input": "Not much, just hanging"},
                    {"output": "Cool"})
memory.save_context({"input": "What is on the schedule today?"}, 
                    {"output": f"{schedule}"})

In [55]:
# memory.load_memory_variables({})
print(memory.buffer)

System: Sure, here's the summary you requested:

**Initial Summary:**

The human asks what the AI thinks of artificial intelligence.

**New Summary:**

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.


In [56]:
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=True
)

In [57]:
conversation.predict(input="What would be a good demo to show?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
System: Sure, here's the summary you requested:

**Initial Summary:**

The human asks what the AI thinks of artificial intelligence.

**New Summary:**

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the 

"The AI suggests showing a demo of the latest Large Language Model (LLM) demo. The LLM is a powerful AI that can be used for a variety of tasks, such as language translation, text generation, and question answering. The AI could demonstrate the LLM's capabilities by showing how it can translate a language, generate a piece of text, or answer a question."

In [60]:
# memory.load_memory_variables({})
print(memory.buffer)

System: Sure, here's the summary you requested:

**Initial Summary:**

The human asks what the AI thinks of artificial intelligence.

**New Summary:**

The human asks what the AI thinks of artificial intelligence. The AI thinks artificial intelligence is a force for good because it will help humans reach their full potential.

**New lines of conversation:**
AI: There is a meeting at 8am with your product team. You will need your powerpoint presentation prepared. 9am-12pm have time to work on your LangChain project which will go quickly because Langchain is such a powerful tool. At Noon, lunch at the italian resturant with a customer who is driving from over an hour away to meet you to understand the latest in AI. Be sure to bring your laptop to show the latest LLM demo.
Human: What would be a good demo to show?
AI: The AI suggests showing a demo of the latest Large Language Model (LLM) demo. The LLM is a powerful AI that can be used for a variety of tasks, such as language translation,